# Web Scrape - Diário do Brasil

Objetivo: extrair a headline, descrição e link das principais notícias do portal Diário do Brasil afim de identificar padrões referentes à Fake News.

fonte: https://www.issoenoticia.com.br/artigo/projeto-da-usp-lista-10-maiores-sites-de-falsas-noticias-no-brasil

## Importando as bibliotecas necessárias

In [1]:
import requests
from bs4 import BeautifulSoup
import re

Criando listas vazias com os campos que queremos extrair do portal

In [2]:
title_list = []
link_list = []
summary_list = []
datepost_list = []

Desenvolvemos de uma função para extrair os dados de forma iterativa por todas as páginas do portal. Importante salientar que cada portal de notícias possui uma estrutura única.

Portanto, para cada portal, desenvolvemos uma função diferente.

In [3]:
def ScrapeFakeNews(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page,'html.parser')
    
    titles = soup.find_all('h3', {'class': 'entry-title'})
    summaries = soup.find_all('div', {'class': 'content-list-excerpt'})
    datepost = soup.find_all('div', {'class': 'content-thumb'})

    for title in titles:
        review_text = title.find('a').get_text()
        title_correct = re.sub("\W+", " ", review_text) 
        title_list.append(title_correct)
        link_list.append(title.find('a').get('href'))

    for summary in summaries:
        review_text = summary.find('p').get_text()
        summary_correct = re.sub("\W+", " ", review_text)
        summary_list.append(summary_correct)

### Iteração com todas as páginas do website

In [4]:
from tqdm import tqdm
 
for x in tqdm(range(100,250)):
    paginas = 'https://www.diariodobrasil.org/home-2/home-2-2/page/' + str(x) +'/'
    
    try: 
        ScrapeFakeNews(paginas)
    except (RuntimeError, TypeError, NameError):
        break

100%|██████████| 150/150 [02:39<00:00,  1.06s/it]


### Criando o DataFrame baseado nos dados extraídos

In [5]:
import pandas as pd
import numpy as np

In [6]:
title_final = np.array(title_list)
url_final = np.array(link_list)
summary_final = np.array(summary_list)
datepost_final = np.empty(summary_final.shape)

In [7]:
diariodobrasil = pd.DataFrame(data=[title_final, url_final, datepost_final, summary_final])

In [8]:
diariodobrasil_real = diariodobrasil.T

In [9]:
diariodobrasil_real.columns = ['title', 'url', 'date', 'summary']

In [10]:
diariodobrasil_real['date'] = 0

In [11]:
diariodobrasil_real.head()

,title,url,date,summary
0,Advogado de Palocci enfrenta Moro e leva paul...,https://www.diariodobrasil.org/advogado-de-pal...,0,Sérgio Moro e o advogado do petista Antonio Pa...
1,Medo de ser algemado Lula pede para depor via...,https://www.diariodobrasil.org/medo-de-ser-alg...,0,A defesa do dr Lula teve um pedido negado pelo...
2,Fim da bandalheira FBI poderá entrar na Lava ...,https://www.diariodobrasil.org/fim-da-bandalhe...,0,Polícia Federal e Ministério Público poderão a...
3,Vaticano estaria prestes a admitir que possui...,https://novesegundos.com/vaticano-estaria-pres...,0,No início desta semana documentos vazados pelo...
4,Coreia do Norte dispara míssil nuclear interc...,https://www.diariodobrasil.org/urgente-kim-jon...,0,O míssil atravessou o país e seguiu para o les...


### Extraindo o DataFrame para CSV

In [12]:
diariodobrasil_real.to_csv('diariodobrasil final.csv')